### Предсказание количества ретвитов

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.decomposition import TruncatedSVD

from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

from sklearn.linear_model import LinearRegression,Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

import torch

import string
import re

from tqdm import tqdm, trange, tqdm_notebook

from nltk.tokenize import TweetTokenizer
import string

import warnings
warnings.filterwarnings('ignore')

import random
import pymorphy2

from collections import Counter


import matplotlib.pyplot as plt
%matplotlib inline


#### Зафиксируем seed для воспроизводимости

In [2]:
#фиксируем seed 
def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True

init_random_seed(value=1712)

База данных и таблицы состоит из 12 столбцов. <br>

***Описание столбцов***:
 - **id**: уникальный номер сообщения в системе twitter;
 - **date**: дата публикации сообщения (твита);
 - **name**: имя пользователя, опубликовавшего сообщение;
 - **text**:  текст сообщения (твита);
 - **type**: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
 - **rep**: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
 - **rtw**: количество ретвитов к данному сообщению (количество копирований этого сообщения другими пользователями);
 - **faw**: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
 - **stcount**: число всех сообщений пользователя в сети twitter;
 - **foll**: количество фоловеров пользователя (тех людей, которые читают пользователя);
 - **frien**: количество друзей пользователя (те люди, которых читает пользователь);
 - **listcount**: количество листов-подписок в которые добавлен твиттер-пользователь.

In [3]:
n_columns = ['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_pos = pd.read_csv('data/positive.csv', sep=';', names=n_columns)
data_neg = pd.read_csv('data/negative.csv', sep=';', names=n_columns)

In [4]:
data_pos.head()

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [5]:
data_neg.head()

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [6]:
df = data_pos.append(data_neg)

перевод даты в удобный формат для просмотра

In [7]:
import datetime
df['date_comf'] = df.date.apply(lambda x: datetime.datetime.fromtimestamp(x))

In [8]:
df.date_comf.min(),df.date_comf.max()

(Timestamp('2013-12-06 14:32:07'), Timestamp('2014-01-20 09:31:54'))

In [9]:
df.sort_values(by=['rtw'], ascending=False).head(20)

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf
104932,411125460018683904,1386854922,angelinagunbina,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13817,0,7005,1719,1887,1,2013-12-12 17:28:42
80674,410748015344091136,1386764932,Jake_and_Cake,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13793,0,11857,526,254,9,2013-12-11 16:28:52
80624,410747573662523392,1386764827,rainydamon,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13791,0,2047,769,680,0,2013-12-11 16:27:07
79028,410727313773723648,1386759997,run_fools,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13789,0,751,59,75,0,2013-12-11 15:06:37
75569,410648531922616320,1386741214,EshikFromMars,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13762,0,12099,156,93,2,2013-12-11 09:53:34
75383,410646291509940224,1386740680,KillForRevenge,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13759,0,947,123,262,0,2013-12-11 09:44:40
75121,410643066551140353,1386739911,DreamerOcean,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13747,0,3449,135,135,0,2013-12-11 09:31:51
75086,410642715140169728,1386739827,JoFromMars,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13745,0,30129,773,684,10,2013-12-11 09:30:27
75028,410642027433717760,1386739663,AMuHkocbka,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13742,0,13701,124,89,5,2013-12-11 09:27:43
74556,410636228170907648,1386738280,KochetkovaDi,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13733,0,28943,1324,426,22,2013-12-11 09:04:40


In [10]:
df.sort_values(by=['rtw'], ascending=False).iloc[0].text

'RT @JaredLeto: Моя русская семья, я скучаю по вам! До скорых встреч. Сохраните немного снега для меня, и не дайте ему растаять\xa0;)'

In [11]:
df[df.name == 'JaredLeto']

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf


##### как видим есть посты, которые были ретвитнуты, но первоначального твита в датасете нет

In [12]:
cnt_rtw = df.rtw.value_counts()

In [13]:
print(f'кол-во твитов, которые ретвитнули = {sum(cnt_rtw[cnt_rtw.index>0])}| кол-во твитов без ретвитов = {cnt_rtw[cnt_rtw.index==0].values[0]}')

кол-во твитов, которые ретвитнули = 36930| кол-во твитов без ретвитов = 189904


In [14]:
sum(cnt_rtw[cnt_rtw.index>0])/df.shape[0]

0.16280628124531596

##### как видим выборка несбалансированная + часть данных таргета будем исключать

In [15]:
print(f'кол-во всего текстов = {df.shape[0]}| # уникальных = {df.text.nunique()}| дельта = {df.shape[0]-df.text.nunique()}')

кол-во всего текстов = 226834| # уникальных = 217440| дельта = 9394


In [16]:
df[df.text.duplicated()].sort_values(by=['rtw'], ascending=False)

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf
104932,411125460018683904,1386854922,angelinagunbina,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13817,0,7005,1719,1887,1,2013-12-12 17:28:42
80674,410748015344091136,1386764932,Jake_and_Cake,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13793,0,11857,526,254,9,2013-12-11 16:28:52
80624,410747573662523392,1386764827,rainydamon,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13791,0,2047,769,680,0,2013-12-11 16:27:07
79028,410727313773723648,1386759997,run_fools,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13789,0,751,59,75,0,2013-12-11 15:06:37
75569,410648531922616320,1386741214,EshikFromMars,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13762,0,12099,156,93,2,2013-12-11 09:53:34
75383,410646291509940224,1386740680,KillForRevenge,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13759,0,947,123,262,0,2013-12-11 09:44:40
75121,410643066551140353,1386739911,DreamerOcean,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13747,0,3449,135,135,0,2013-12-11 09:31:51
75086,410642715140169728,1386739827,JoFromMars,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13745,0,30129,773,684,10,2013-12-11 09:30:27
75028,410642027433717760,1386739663,AMuHkocbka,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13742,0,13701,124,89,5,2013-12-11 09:27:43
74556,410636228170907648,1386738280,KochetkovaDi,"RT @JaredLeto: Моя русская семья, я скучаю по ...",1,0,13733,0,28943,1324,426,22,2013-12-11 09:04:40


In [17]:
most_common_text = df.text.value_counts()

In [18]:
most_common_text[most_common_text >2]

Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!                                                  156
RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT                                                                                                                 105
RT @Creative_Link: Дождались! Официальный трейлер "Иван Царевич и Серый Волк - 2". Смотреть всем: http://t.co/gmelSGMIuW Толи еще будет)                                 74
sm&amp;gt; напомни, как называется: фотка, вокруг черная рамка, а внизу - надпись?\nau&amp;gt; некролог?\nsm&amp;gt; не.. (\nau&amp;gt; демотиватор?\nsm&amp;gt; ДА!     70
RT @Podslyshano: люблю зиму за то, что люди носом разблокируют телефоны, и такие оглядываются типа без палева)) а я то все вижу. да да.                                  53
RT @Creative_Link: Иван Царевич и Серый Волк - 2. Идем в кино? премьера 26 декабря))) http://t.co/gmelSGMIuW                                

In [19]:
tweet_ARTEM_KLYUSHIN =  df[df.text.str.contains('Кто ненавидит пробки ретвит')]
tweet_ARTEM_KLYUSHIN

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf
12791,410789572235505664,1386774840,svetila89,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,108,0,24,13,1720,0,2013-12-11 19:14:00
12792,410789574848548864,1386774841,Pyziee,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,108,0,34,5,1940,0,2013-12-11 19:14:01
12793,410789574999547904,1386774841,IlonaViluma,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,108,0,60,40,2003,0,2013-12-11 19:14:01
12794,410789576266248192,1386774841,nikkierlendson,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,108,0,4,0,744,0,2013-12-11 19:14:01
12795,410789619232677888,1386774852,mazhitnezhe,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,172,0,52,4,2002,0,2013-12-11 19:14:12
12796,410789619786321920,1386774852,Gulzi89_09,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,172,0,10,3,1577,0,2013-12-11 19:14:12
12797,410789621501792256,1386774852,nesajbenedict,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,172,0,38,4,742,0,2013-12-11 19:14:12
12798,410789621715730433,1386774852,koeizwvxa,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,172,0,563,6,2001,0,2013-12-11 19:14:12
12799,410789621875093504,1386774852,muha_03,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,172,0,5,1,767,0,2013-12-11 19:14:12
12800,410789665474899969,1386774863,18Renatochka,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,-1,0,232,0,222,14,1077,0,2013-12-11 19:14:23


In [20]:
tweet_ARTEM_KLYUSHIN.date_comf.min(),tweet_ARTEM_KLYUSHIN.date_comf.max()

(Timestamp('2013-12-11 19:14:00'), Timestamp('2013-12-11 19:21:47'))

In [21]:
tweet_ARTEM_KLYUSHIN.rtw.min(),tweet_ARTEM_KLYUSHIN.rtw.max()

(108, 1381)

In [22]:
tweet_check =df[df.text.str.contains('однобокость силовых структур иногда поражает')]
tweet_check

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf
83726,420069554807517184,1388987361,bentmathepo,однобокость силовых структур иногда поражает. ...,-1,0,0,0,3,2,0,0,2014-01-06 09:49:21
83759,420071989315772416,1388987941,ushakoova,однобокость силовых структур иногда поражает. ...,-1,0,0,0,668,2149,74,0,2014-01-06 09:59:01
83781,420073437197905920,1388988286,censojuls,однобокость силовых структур иногда поражает. ...,-1,0,0,0,678,2105,45,0,2014-01-06 10:04:46
83790,420073595742601217,1388988324,chelnakovasasha,однобокость силовых структур иногда поражает. ...,-1,0,0,0,594,2108,45,0,2014-01-06 10:05:24
83793,420073855458111489,1388988386,hoffnigopan,однобокость силовых структур иногда поражает. ...,-1,0,0,0,22,66,71,0,2014-01-06 10:06:26
83794,420073899401834496,1388988396,ivan_polushin,однобокость силовых структур иногда поражает. ...,-1,0,0,0,641,2107,46,0,2014-01-06 10:06:36
83797,420074155048857600,1388988457,skeezdaridi,однобокость силовых структур иногда поражает. ...,-1,0,0,0,24,74,111,0,2014-01-06 10:07:37
83848,420077420004139008,1388989236,feckcococi,однобокость силовых структур иногда поражает. ...,-1,0,0,0,28,63,91,0,2014-01-06 10:20:36
83850,420077613449629696,1388989282,katostrovskayaa,однобокость силовых структур иногда поражает. ...,-1,0,0,0,685,2099,53,0,2014-01-06 10:21:22
83852,420077795704725504,1388989325,evalgakum,однобокость силовых структур иногда поражает. ...,-1,0,0,0,5,0,0,0,2014-01-06 10:22:05


In [23]:
tweet_check.date_comf.min(),tweet_check.date_comf.max()

(Timestamp('2014-01-06 09:49:21'), Timestamp('2014-01-06 11:26:26'))

In [24]:
tweet_check.rtw.min(),tweet_check.rtw.max()

(0, 1)

Как видим в случае дубликатов:
 1. мб ретвит какого-то известного человека и исходный ретвит будет отсутствовать в нашей выборке.
 2. мб одинаковый текст твитов, но они не являются ретвитами одного

С учётом того, что нужно избегать утечек данных  **в качестве входных данных используем те данные, которые доступны пользователю перед публикацией твита.**

## Создание выборки:
 1. Пользуемся только теми твитами, которые не являются ретвитами.
  - Проверяем что данный твит - действительно ретвит (наличие @ и рт/rt, а не призыв в твите к ретвиту)
  - - можем использовать не только текстовые признаки, но и признаки автора и дату 
 2. Если был ретвит твита,
  - и у пользователя с никнеймом исходного твита есть данное сообщение, то добавить ему максимальный по значению ретвит
     - - можем использовать не только текстовые признаки, но и признаки автора и дату
  - и исходного твита нет в нашей выборке, то добавляем этот твит и назначаем ему последнее значение ретвитов по последней дате (из всех твитов).
      - - можем использовать признаки только из текста твита (дату из других твитов назначать плохо, тк не всегда есть первые ретвиты исходного твита) 

### Выборка 1

посмотрим твиты,которые начинаются со слова rt 

In [25]:
cnt = 0 
for num,x in enumerate(df[df['text'].str.lower().str.startswith(' rt') |df['text'].str.lower().str.startswith('rt') |df['text'].str.lower().str.startswith('рт') ].iterrows()):
    if x[1]['text'].lower()[:3] !='rt!' and x[1]['text'].lower()[:3] !='rt ':
        cnt+=1
        if cnt == 20:
            break
        print(x[1]['text'])
        print('---')
        
cnt

RT‏ ‏@Raven_D1 Рисунок без названия. А я назову его: СНЕЖНАЯ КОРОЛЕВА. ))) 
© http://t.co/xBh3lyxRVH
---
рт если не проебал этот момент) http://t.co/BzCyYEw0c0
---
RT"@fardyanas: Wah au blg apa?:|"@aulyarachmaa: Молодёжка 38 серия смотреть онлайн http://t.co/2iTxYRVjVm""
---
RT: местная @mokrenusha Защищённые твиты 2 мин Оля не хочет идти на сделку , на услугу за услугой , а так не честненько :(
---
RT"@MrPushisty: Деда Мороза, умершего на корпоративе в Якутске, могла задушить подруга" - Снегурочка вроде раньше внучкой была :(
---
RT@CIO_Optimal Депутатов воронежской Гордумы могут лишить зарплаты http://t.co/ndUdU16f4r …
---
РТ, кому не нужен завтра калькулятор?Я свой благополучно потеряла,из-за чего очень расстроилась,я была к нему крайне привязана:(
---
РТ через 2 дня! :(
Биологииия – страшная штука :с
---


8

In [26]:
cnt = 0 
for num,x in enumerate(df[df['text'].str.lower().str.startswith(' rt') |df['text'].str.lower().str.startswith('rt') ].iterrows()):
    text = x[1]['text'].lower()
    text = text.split('rt')
    
    try:
        next_word = text[1].split()[0]
        if '@' not in next_word:
            cnt+=1
            if cnt == 100:
                break
            print(next_word,'|',x[0],'|',x[1]['text'])
            print('---')
    except:
        print('EROR|',x[0],'|',x[1]['text'])
        break

если | 18305 | Rt если ты подписан на мой канал ;) http://t.co/5SqAkcDjdn
---
хорошая | 40596 | RT хорошая шутка) @BombayDancer Ребята, у кого есть дома лишняя комната жилая?
---
от | 42213 | RT от Юрчика,нежданно -негаданно)) http://t.co/wKkuZS7jkK
---
косяк | 46794 | RT косяк @kyrochkina2 Защищённые твиты 14 ч @mokrenusha  с ДР ХАААЧ:*
---
бррр, | 47106 | RT бррр, страшно) @AnnaStinktIn "@interesshtuki: Отличная идея! http://t.co/jIXLe7FNUV" ужас))
---
‏ | 47164 | RT‏ ‏@Raven_D1 Рисунок без названия. А я назову его: СНЕЖНАЯ КОРОЛЕВА. ))) 
© http://t.co/xBh3lyxRVH
---
! | 55809 | RT! #ТВИТТЕР в реальной жизни! :D  #ГераСтрейзанд @HerStreis #стрейзер http://t.co/YOhr1Tf0Wq
---
всякоразные) | 79483 | RT всякоразные) @cruiserom Кстати, а какие у АКБ привилегии? Ну там, может, бесплатный проезд в метро или в мавзолей без очереди, волнуюсь))
---
! | 88991 | RT! #ТВИТТЕР в реальной жизни! :D  #ГераСтрейзанд @HerStreis: http://t.co/sevAlcT2uL
---
! | 114180 | RT! :-) @FindJoeAtEaster: Зөөхий

In [27]:
cnt

12

для данного примера, возникают проблемы из-за лишних символов

In [28]:
df.iloc[47164].text

'RT\u200f \u200f@Raven_D1 Рисунок без названия. А я назову его: СНЕЖНАЯ КОРОЛЕВА. ))) \n© http://t.co/xBh3lyxRVH'

In [29]:
print(df.iloc[47164].text)

RT‏ ‏@Raven_D1 Рисунок без названия. А я назову его: СНЕЖНАЯ КОРОЛЕВА. ))) 
© http://t.co/xBh3lyxRVH


#### как видим rt / rt! вначале предложения хорошо показывает что сообщение было ретвитнуто


посмотрим твиты,которые заканчиваются словами rt и rt! 

In [30]:
cnt = 0 
for num,x in enumerate(df[df['text'].str.lower().str.endswith('rt') |df['text'].str.lower().str.endswith('rt!') ].iterrows()):
    check = x[1]['text'].lower().split()[-1]
    if (check =='rt' or (check =='#rt' or check[-3:] =='#rt')) or check =='rt!'or check =='#rt!': #and check[-4:] !='\nrt'
        cnt+=1
        if cnt == 100:
            break
        print(x[0],x[1]['text'],'|',check)
        print('---')
    elif 'http' in check: # случаи когда нормальные записи
        continue
    else:  # случаи когда нормальные записи
        continue

1190 Вах... 5000 подписчиков... Круто!!! :-D Всем спасибо! Оставайтесь с нами, приглашайте друзей на @vsmexe Еееее... RT | rt
---
4027 RT @artemblayn: 20 фолловеров еще и 1000 ыхых
Поможете?) 
#RT | #rt
---
4665 RT @95Shevchenko: МНЕ НАДО ЕЩЁ 56 ФОЛЛОВЕРОВ))
ЛЕЕЕЕНТАААА ПОМОГИТЕ #RT | #rt
---
4667 RT @95Shevchenko: МНЕ НАДО ЕЩЁ 56 ФОЛЛОВЕРОВ))
ЛЕЕЕЕНТАААА ПОМОГИТЕ #RT | #rt
---
7085 И 30 Second To Mars обходит Бибера в актуальных темах)) Вот они реальные #mtvstars. Голосуй за марсов!!! #RT | #rt
---
15552 RT @Serbolov_darhan: друзья, ГОЛОСУЕМ за @aliokapovа всего два дня остался) позвоните по НОМЕРУ 7045 и нажмите число 11... RT | rt
---
17122 Кто нибудь сейчас хочет Твит со случайной картинкой? :D
#RT #RT | #rt
---
19559 RT @KateKatya_pop: Кому ещё посвятить 19.300 твит?) #RT | #rt
---
19604 RT @KateKatya_pop: Кому ещё посвятить 19.300 твит?) #RT | #rt
---
20531 Наращивание ресниц:* Скидка 63% на наращивание ресниц любой сложности 1100 р. Снятие ресни http://t.co/CgdUK5OZpZ #RT | #rt


уберем твиты, которые начинаются с rt (являются потенциальными ретвитами), и посмоторим какие твиты заканчиваются на rt/rt! 

In [31]:
cnt = 0 
for num,x in enumerate(df[(~df['text'].str.lower().str.startswith(' rt') & ~df['text'].str.lower().str.startswith('rt') ) & ( df['text'].str.lower().str.endswith('rt') |df['text'].str.lower().str.endswith('rt!') )].iterrows()):
#     check = x[1]['text'].lower()[-3:]
    check = x[1]['text'].lower().split()[-1]
    cnt+=1
    if cnt == 100:
        break
    print(x[0],x[1]['text'],'|',check)
    print('---')

1190 Вах... 5000 подписчиков... Круто!!! :-D Всем спасибо! Оставайтесь с нами, приглашайте друзей на @vsmexe Еееее... RT | rt
---
4864 И да, Даня, с днем рождения! И от всего зверского джаза тоже!))) @amatorystewart | @amatorystewart
---
5311 Тульский вице-губернатор извинилась за подзаборных бабок"/Это самое простое: пугнула,а вид сделала-кашель))) http://t.co/g7lTQhEYRt | http://t.co/g7ltqheyrt
---
7085 И 30 Second To Mars обходит Бибера в актуальных темах)) Вот они реальные #mtvstars. Голосуй за марсов!!! #RT | #rt
---
10338 У меня есть упоротый халат с овечками, ахах))) #webcamtoy http://t.co/DIhMu5v3Rt | http://t.co/dihmu5v3rt
---
11318 Сегодня полгода, как мы с любимой познакомились))) GIVE_HEART | give_heart
---
13550 Быстрей бы уже каточееек!со всеми очень хочется увидится) как же я по всем соскучилась! http://t.co/6Sy4ic2Frt | http://t.co/6sy4ic2frt
---
14119 ахах, что мне скинули, ну неет я не такая:D http://t.co/XU59dDAKrt | http://t.co/xu59ddakrt
---
17122 Кто нибудь сейчас

In [32]:
cnt

85

##### Как видим если в конце есть просьба ретвитнуть (а вначале отсутствует идентификатор ретвита), то твит не является ретвитом

посмотрим твиты, которые не начинаются и не заканчиваются словами rt/rt! , но содержат данные токены внутри сообщения

In [33]:
cnt = 0 
for num,x in enumerate(df[(~df['text'].str.lower().str.startswith(' rt') & ~df['text'].str.lower().str.startswith('rt') & ~df['text'].str.lower().str.endswith('rt') & ~df['text'].str.lower().str.endswith('rt!')) & ( df['text'].str.lower().str.contains('rt!')|df['text'].str.lower().str.contains(' rt'))].iterrows()):
    check = x[1]['text'].lower().split()[-1]
    cnt+=1
    if cnt <= 100:
        print(x[0],x[1]['text'],'|',check)
        print('---')

1450 :)) RT @FoolsNobodys дуэт "Не пара" http://t.co/nV0p44hRcM | http://t.co/nv0p44hrcm
---
1843 и чо? 4.4 адаптирован для смартов с 512мб оперативы) RT @vadimrerih @strelec6120 бля у мну то оперативы 1 гик | гик
---
2104 Фубляя) RT @ZhuravlevPavel: Луческу сделают тренером сборной через Попеску | попеску
---
3177 #голос я за. Тину короче! Она-самая самая крутая исполнительница на проекте! Это не обсуждается! Кто согласен, rt) | rt)
---
4938 Ціхо RT @pinksyrop: @TvojaMalanka бо ти ще маленька)) виростеш - вшариш | вшариш
---
5216 Вот точно! RT @maskuznetsova: Сегодня игра ЦСКА против СКА понравилась! Всегда бы так играли:)) | играли:))
---
5758 Это #успех, ящетаю ))) #насебяпосмотридура RT @Vitalij51: @twitchenko мудакты. Чесслово. | чесслово.
---
7448 Кто последний - тот и папа ;) RT @nuuffsaiid: Дети О_О а я не в курсах?!? Непорядок! | непорядок!
---
8674 Выедает моск!:) RT @ukrpravda_news В Тайвані зняли мультик про те, як Янукович поїхав в Китай! http://t.co/vNN3jUhjRB #євромайдан

In [34]:
cnt

275

Тут можно заметить что идёт комментарий до ретвитнутого сообщения.

Я считаю, что такие сообщения не являются нашей исходной выборкой и поэтому отношу их к ретвиту, однако если их ретвитнули с комментарием, то я считаю данное сообщение твитом.

В итоге можно относить к ретвиту следующие записи: 
 - **есть слово RT и после него идёт никнейм**


Посмотрим на обратные случаи (когда у нас твит не является  ретвитом по нашему правилу)

In [35]:
cnt = 0 
for num,x in enumerate(df[df['text'].str.lower().str.contains('rt')].iterrows()):
    if 'rt! ' in x[1]['text'].lower():
        identif = 'rt!'
    else:
        identif = 'rt'
    text = x[1]['text'].lower().split(identif)
    try:
        if '@' not in text[1].split()[0]:
            cnt+=1
            if cnt == 100:
                break
            print(text[1].split()[0],'|',x[0],'|',x[1]['text'])
            print('---')
    except:
        print('EROR|',x[0],'|',x[1]['text'])
        break

_sk | 101 | @redisskakat @girt_sk Единственное,что меня расстраивает на Бали. Невозможность сходить в кинотеатр с друзьями на интересный фильм :)
---
ushkan | 153 | @sailortushkan боюсь пересказывать это было бы сложнее)
---
wla | 162 | Вот чем я пахну после ванны)#запахсчастья http://t.co/9McjwrtwLa
---
ain | 196 | @koortain @veshchij_oleg я так понял, что будут решать или в запас или комиссовать)))
---
ovsky | 230 | малышка,сегодня будем веселиться.потому уже просто ну очень надо.жду вечера с @yulisukkyb.надеюсь, что Котяра доедет до нас)@nikitamartovsky
---
69 | 258 | @ProSport69 нафиг такую жизнь!Имя другое взяла и зачем только придумала о себе историю,я не знаю.Надо б ей написать,да на землю приспустить)
---
y | 286 | Гороскоп для ВСЕХ на выходные. Смотреть видео :) / #party #hustle #hustlehard http://t.co/j11WUHxz6O
---
inside | 314 | @lionheartinside иди против них!бунтуй!ложись спать)))
---
ux90 | 461 | За эту неделю наприходило :) http://t.co/ZTraRTuX90
---
in | 468 | @PRETTY_

Посмотрим на целевые случаи (когда по нашему правилу твит является ретвитом)

In [36]:
cnt = 0 
retweet_ids = {}
for num,x in enumerate(df[df['text'].str.lower().str.contains('rt')].iterrows()):
    if 'rt! ' in x[1]['text'].lower():
        identif = 'rt!'
    else:
        identif = 'rt'
    text = x[1]['text'].lower().split(identif)
    try:
        if '@' in text[1].split()[0]:
            cnt+=1
            retweet_ids[x[1]['id']] = 1
            if cnt < 100:
#                 break
                print(text[1].split()[0],'|',x[0],'|',x[1]['text'])
                print('---')
    except:
        print('EROR|',x[0],'|',x[1]['text'])
#         break

@katiacheh: | 2 | RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!
---
@digger2912: | 3 | RT @digger2912: "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…
---
@spoonlamer: | 6 | RT @SpoonLamer: Ох,900 :D ну это конечно же @twinkleAYO . Чтобы у нее было много друзей, ведь она такая мимими &lt;3
---
@veregijytaqo: | 7 | RT @veregijytaqo: У тебя есть ухажёр? Нет - мои уши не кто не жрёт :D
---
@dicyziqecida: | 11 | RT @dicyziqecida: Как-то я забыла, что вчера приехал из деревни наш котэня и испугалась, когда он утром пришел ко мне общаться. :) Доброе у
---
@abdullin_a_r: | 16 | RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГАТУ о контрольных предупреждают уже на контрольной)
---
@lumenso: | 23 | RT @lumenso: @GruzdevVladimir @Sergey_Degtyrev 1) При поддержке администрации МО Белевский район сборная Белевского района по http://t.co/r…
---
@lubalubochka2: | 30 | RT @Lubalubochka2: @edvard146 для тебя;) http://t.

EROR| 11318 | Сегодня полгода, как мы с любимой познакомились))) GIVE_HEART
EROR| 13550 | Быстрей бы уже каточееек!со всеми очень хочется увидится) как же я по всем соскучилась! http://t.co/6Sy4ic2Frt
EROR| 14119 | ахах, что мне скинули, ну неет я не такая:D http://t.co/XU59dDAKrt
EROR| 20531 | Наращивание ресниц:* Скидка 63% на наращивание ресниц любой сложности 1100 р. Снятие ресни http://t.co/CgdUK5OZpZ #RT
EROR| 22444 | Оу, Лимон танцует танго :)) Принц Лимон- Денис Савин. Графиня Вишня- Анна Леонова. anya_leo #cipollino… http://t.co/56wu7OC3Rt
EROR| 23637 | всегда смеялась в этот момент) http://t.co/vqqDrObURT
EROR| 26281 | Скинь фотку, кто лайкнет, задай тот же вопрос) с: —  http://t.co/I1aur6voRt
EROR| 28295 | Рано утром на рассвете заглянул в соседний двор, там бабуся на мопеде разъе*ала весь забор! :D 0 http://t.co/zvsuzXjgrT
EROR| 28636 | #Администратор кафе похвастался смартфоном от Apple в вишневом саду. #улыбнуло ;) #twitRT
EROR| 46309 | Это моя новая эстонская шапочка:-) 

In [37]:
cnt

36866

In [38]:
all_cnt = df[df['text'].str.lower().str.contains('rt')].shape[0]
print(f'кол-во всех твитов, в которых есть rt = {all_cnt} | кол-во твитов rt,кот-ые явл-ся ретвитом {cnt} | дельта = {all_cnt-cnt}')

кол-во всех твитов, в которых есть rt = 39467 | кол-во твитов rt,кот-ые явл-ся ретвитом 36866 | дельта = 2601


In [39]:
len(retweet_ids)

36866

#### Создание выборки

In [40]:
df['retweet'] = df.id.apply(lambda x: 1 if x in retweet_ids else 0 )

In [41]:
df.head(10)

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,2013-12-06 14:32:07,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,2013-12-06 14:32:07,0
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,2013-12-06 14:32:07,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,2013-12-06 14:32:07,1
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,2013-12-06 14:32:23,0
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0,2013-12-06 14:32:23,0
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2,2013-12-06 14:32:23,1
7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1,0,2,0,393,112,153,0,2013-12-06 14:32:24,1
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0,0,0,5872,1387,1431,12,2013-12-06 14:32:24,0
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,0,0,0,263,5,36,0,2013-12-06 14:32:24,0


In [42]:
df.retweet.value_counts()

0    189968
1     36866
Name: retweet, dtype: int64

In [43]:
df[df.retweet == 0].rtw.value_counts()

0     189325
1        637
6          2
2          2
14         1
4          1
Name: rtw, dtype: int64

In [44]:
for num,x in enumerate(df[(df.retweet == 0) & (df.rtw >= 1)].iterrows()):
    if num == 20:
        break
    print(x[0],'|',x[1]['rtw'],'|',x[1]['text'],)
    print('---')

211 | 1 | @shitpisspuke @CKAHAX да норм, еще подрастет, не буду его гномом считать)))
---
462 | 1 | @Aprelushka видно что в вечернем платье ты себя намного уютней чувствовала))) ты молодец!
---
1589 | 1 | @valinevichka не стоит благодарности!!!я же Богиня блять, должна помогать)))))):D
---
2109 | 1 | @Belieber_Ann1 а ты мой =*
самый упоротый любимый твиттерский :D
---
3195 | 1 | Противоположности притягиваются, ага. Мы с @Lexieside дополняем друг друга)
---
4453 | 1 | "@owredxw: Эт я с аней)))))))))) http://t.co/fdaJCPERVA"идиотина,вот это и распечатаю
---
4685 | 1 | @vodkainmyblood @alien_in_me Мне тоже очень нравится :D
---
5534 | 1 | @yana_vyazevich @Maksimova09 согласна.нет парня-нет проблем,Морозова)
---
5723 | 1 | @OAULK @KATYA_lovedance присоединяюсь:*
С днем рождения:*
---
6348 | 1 | @Fridma_n остальные не достойны :D я же в тви почти не сижу :)
---
7052 | 1 | @orange_kaktus Спасибо тебе большое) Ты самый лучший хозяин, которого только пожелать можно)**
---
7839 | 1 | а че тогд

посмотрел часть из них вручную, данные твиты не являются ретвитами либо нет такого пользователя

In [45]:
df1 = df[df.retweet == 0].copy()

In [46]:
df1.shape

(189968, 14)

In [47]:
df1.rtw.value_counts()

0     189325
1        637
6          2
2          2
14         1
4          1
Name: rtw, dtype: int64

In [48]:
df1.text.value_counts()

Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!                                                  156
sm&amp;gt; напомни, как называется: фотка, вокруг черная рамка, а внизу - надпись?\nau&amp;gt; некролог?\nsm&amp;gt; не.. (\nau&amp;gt; демотиватор?\nsm&amp;gt; ДА!     70
однобокость силовых структур иногда поражает. Очевидные вещи им не видны.( //u.to/ardwBQ #сми Санкт-Петербург Новый                                                      27
Сегодня ходил отрабатывать два пропуска по физре, первой парой и третьей) Вобщем в конце первой пары препод сказал:"А должники остаются!"...                             24
настроение вроде ничего, а вот  погодка портится...(                                                                                                                     18
надо создать акк секси ноги майкрофта холмса(                                                                                               

Не вижу смысла удалять дублирования, тк они отражают насколь твит возможно ретвитнуть или нет (это мб как Positive, так и Negative sampling)

### Выборка 2

Цель: пополнить прошлую выборку позитивными примерами.

Способ: Если был ретвит твита,
  - и у пользователя с никнеймом исходного твита есть данное сообщение, то добавить ему максимальный по значению ретвит
     - - можем использовать не только текстовые признаки, но и признаки автора и дату
  - и исходного твита нет в нашей выборке, то добавляем этот твит и назначаем ему последнее значение ретвитов по последней дате (из всех твитов).
      - - можем использовать признаки только из текста твита (дату из других твитов назначать плохо, тк не всегда есть первые ретвиты исходного твита) 


In [49]:
df2 = df[df.retweet == 1].copy()

In [50]:
df2

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,2013-12-06 14:32:07,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,2013-12-06 14:32:07,1
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2,2013-12-06 14:32:23,1
7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1,0,2,0,393,112,153,0,2013-12-06 14:32:24,1
11,408906815125340160,1386325956,kazerepiconu,"RT @dicyziqecida: Как-то я забыла, что вчера п...",1,0,4,0,294,87,87,0,2013-12-06 14:32:36,1
16,408906854803451904,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5,2013-12-06 14:32:45,1
23,408906911049068544,1386325979,ElenaChuravkina,RT @lumenso: @GruzdevVladimir @Sergey_Degtyrev...,1,0,3,0,504,39,65,1,2013-12-06 14:32:59,1
30,408906972449492992,1386325994,edvard146,RT @Lubalubochka2: @edvard146 для тебя;) http:...,1,0,1,0,7072,4422,4095,44,2013-12-06 14:33:14,1
35,408907042154635264,1386326010,KristinaMuza,RT @StarScrim3: #RT Электрогенератор http://t....,1,0,20,0,7156,1755,1804,12,2013-12-06 14:33:30,1
38,408907043308068864,1386326010,PankratovaKotya,"RT @amytareva: -слушай, Коть, можно я этим бок...",1,0,1,0,753,72,87,0,2013-12-06 14:33:30,1


(#) Извлекаем никнейм последнего пользователя, которого ретвитнули. 

Не использую весь список никнеймов, т.к. надо было бы долго разворачивать их учёт + когда много пользователй, которых ретвитили в твите, это не является интересующим нас случаем (потому что мы уже работаем с твитом, у которого есть ретвиты, следовательно работаем с "будущим")   

In [51]:
def extract_rt_nikname(text):
    if '@' in text:
        nikname = re.findall('@[^\s]+', text)
        return re.sub('[^a-zA-Zа-яА-Я0-9]+', '', nikname[0])
    else:
        return '!!ERROR!!'

In [52]:
def extract_rt_nikname_all(text):
    result = []
    if '@' in text:
        nikname = re.findall('@[^\s]+', text)
#         print(nikname)
        
        for name in nikname:
            result.append(re.sub('[^a-zA-Zа-яА-Я0-9]+', '', name))
        return result
    else:
        return '!!ERROR!!'

In [53]:
df2['rt_nik'] = df2.text.apply(lambda x: extract_rt_nikname(x))
df2['all_nik_len'] = df2.text.apply(lambda x: len(extract_rt_nikname_all(x)))

In [54]:
df2.head(10)

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet,rt_nik,all_nik_len
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0,2013-12-06 14:32:07,1,KatiaCheh,1
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0,2013-12-06 14:32:07,1,digger2912,1
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2,2013-12-06 14:32:23,1,SpoonLamer,2
7,408906764114206720,1386325944,pycalyruhog,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,1,0,2,0,393,112,153,0,2013-12-06 14:32:24,1,veregijytaqo,1
11,408906815125340160,1386325956,kazerepiconu,"RT @dicyziqecida: Как-то я забыла, что вчера п...",1,0,4,0,294,87,87,0,2013-12-06 14:32:36,1,dicyziqecida,1
16,408906854803451904,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5,2013-12-06 14:32:45,1,AbdullinAR,2
23,408906911049068544,1386325979,ElenaChuravkina,RT @lumenso: @GruzdevVladimir @Sergey_Degtyrev...,1,0,3,0,504,39,65,1,2013-12-06 14:32:59,1,lumenso,3
30,408906972449492992,1386325994,edvard146,RT @Lubalubochka2: @edvard146 для тебя;) http:...,1,0,1,0,7072,4422,4095,44,2013-12-06 14:33:14,1,Lubalubochka2,2
35,408907042154635264,1386326010,KristinaMuza,RT @StarScrim3: #RT Электрогенератор http://t....,1,0,20,0,7156,1755,1804,12,2013-12-06 14:33:30,1,StarScrim3,1
38,408907043308068864,1386326010,PankratovaKotya,"RT @amytareva: -слушай, Коть, можно я этим бок...",1,0,1,0,753,72,87,0,2013-12-06 14:33:30,1,amytareva,2


In [55]:
dict_name = df2.name.value_counts().to_dict()
dict_nikname = df2.rt_nik.value_counts().to_dict()

dict_nikname = {k: v for k, v in sorted(dict_nikname.items(), key=lambda item: item[1],reverse=True)}


In [56]:
dict_nikname

{'TukvaSociopat': 272,
 'palnom6': 140,
 'ARTEMKLYUSHIN': 137,
 'CreativeLink': 121,
 'xeniasobchak': 95,
 'prisoneroO': 90,
 'Podslyshano': 87,
 'wylsacom': 87,
 'lentaruofficial': 63,
 'backto90th': 56,
 'AizaDolmatova': 50,
 'OObnulyay': 49,
 'supremefm': 49,
 'fillipina': 48,
 'radioofmoon': 46,
 'gigliliput': 45,
 'BooOhoo': 43,
 'ktvsktvs': 42,
 'Pavsekakii': 41,
 'tooverkill': 40,
 'DoOrDiexxx': 39,
 'Omega777oleg': 38,
 'DoorDie': 38,
 'CIOOptimal': 37,
 'craazyyymofo': 36,
 'sbezrukov': 35,
 'euromaidan': 35,
 'HrunMorgov': 33,
 'domanep': 33,
 'Dbnmjr': 32,
 'tinakandelaki': 32,
 'ITetereva2': 30,
 'Horansoon': 29,
 'katunder1984': 28,
 'MalishevaE': 28,
 'RadioRecord': 28,
 'adagamov': 27,
 'MrAmirovoy': 26,
 'F1Popov': 26,
 'RussiaMoscow': 26,
 'KateClapp': 26,
 'obnuli': 26,
 'RomaTweetcorn': 25,
 'AlexeyAndronov': 25,
 'tschelovek72': 24,
 'BloodySAW82': 24,
 'TrawkoO': 24,
 'financerf': 23,
 'radioutkin': 23,
 'BigRussianBaws': 23,
 'PsykerO1477': 23,
 'KSHN': 23,
 'Pami

In [57]:
len(dict_name),len(dict_nikname)

(27688, 23104)

количество никнеймов, которых ретвитили и которые есть в нашем датасете

In [58]:
names_intersec = set(dict_name).intersection(set(dict_nikname))
len(names_intersec)

3930

In [59]:
names_intersec = {name:dict_nikname[name] for name in names_intersec}

names_intersec = {k: v for k, v in sorted(names_intersec.items(), key=lambda item: item[1],reverse=True)}

Пример ретвита, который хотим находить

In [60]:
key = 'shagulin'

In [61]:
dict_name[key],dict_nikname[key]

(1, 2)

In [62]:
df2[df2.name == key]

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet,rt_nik,all_nik_len
66481,417252268543864832,1388315667,shagulin,Огонь Олимпиады в Казани :( RT “@Kutilov: http...,-1,0,0,0,8055,1091,446,12,2013-12-29 15:14:27,1,Kutilov,1


In [63]:
for txt in df2[df2.name == key].text:
    print(txt)
    print('---')

Огонь Олимпиады в Казани :( RT “@Kutilov: http://t.co/WUa5RfC0Xy”
---


In [64]:
for txt in df2[df2.rt_nik == key].text:
    print(txt)
    print('---')

RT @shagulin: Огонь Олимпиады в Казани :( RT “@Kutilov: http://t.co/WUa5RfC0Xy”
---
RT @shagulin: Огонь Олимпиады в Казани :( RT “@Kutilov: http://t.co/WUa5RfC0Xy”
---


Как видим исходного пользователя либо самого ретвитнули, либо он прокоментировал и его ретвитнули. 

Данные случаи подходят 

##### 1 вариант  нахождение исходного автора ретвита 

In [65]:
original_authors = {}
for name in tqdm_notebook(names_intersec):
    for txt_name in df2[df2.name == name].text:
        for txt_nik_name in df2[df2.rt_nik == name].text:
            txt_name1 = ' '.join(txt_name.split()[-6:])
            txt_nik_name1 = ' '.join(txt_nik_name.split()[-6:])
            
            if txt_nik_name == txt_name:
                print('equal|',name,txt_name)
                original_authors[name] = txt_name
            elif txt_nik_name1 == txt_name1:
                print('last 6|',name,txt_name1)
                original_authors[name] = txt_name1

last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| dubbcesssecli Самый тяжёлый день- это #четверг (((
last 6| imerkouri Действительно, несколько неожиданное развитие событий:)))) http://t.co/vcpBfnY3tu
last 6| imerkou

Тут можно сделать улучшение воспользоваться расстоянием левенштайна по токенам

добавляем твиты исходного пользователя по id твита

In [66]:
ids_n_rtw = []
ids_n_max_rtw =  []
for name, sent in original_authors.items():
    # пропускаем ретвиты, в которых больше 2 никнеймов
    if len(df2[(df2.name == name)&(df2.all_nik_len <= 2)].text) == 0:
        continue
    print(name)
#     print(sent)
    sent = sent.split()[-6:]
    for txt in df2[(df2.name == name)&(df2.all_nik_len <= 2)].iterrows():
        orig = txt[1]['text'].split()[-6:]
        if orig == sent:
            print(txt[1]['id'],txt[1]['text'])
            ids_n_rtw.append(txt[1]['id'])
        else:
            continue
        print('---')
    # добавляем количество ретвитов от самого нового ретвитнутого сообщения 
    ids_n_max_rtw.append(df2[df2.rt_nik == name].iloc[-1].rtw)
    print(10*'*****')

imerkouri
410483415818199041 Ахахахахаха RT @nzlobin Действительно, несколько неожиданное развитие событий:)))) http://t.co/vcpBfnY3tu
---
**************************************************
wirogeboses
412635825609146368 RT @hofequtamit: — Давай поспорим на $100, кто быстрее скажет алфавит? — Давай! А, б, в, г, д, ... — Алфавит! — Блеать!(
---
**************************************************
mucobyqejeg
411090393829486592 RT @kidukomypolu: хм, включила проигрыватель, первая песня Маси "Мой рай". концерт-это классно)
---
**************************************************
shagulin
417252268543864832 Огонь Олимпиады в Казани :( RT “@Kutilov: http://t.co/WUa5RfC0Xy”
---
**************************************************
reamagcaro
413322070617911297 #RT @scufasaxim : Самый тяжёлый день- это #четверг (((
---
**************************************************
stowveruptcom
413334187073359872 RT @dubbcesssecli: #RT @worldDG : Самый тяжёлый день- это #четверг (((
---
************************

In [67]:
ids_n_max_rtw

[0, 3, 1, 16, 1, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0]

ноль не мб, тк их ретвитнули. Следовательно меняем 0 на 1

In [68]:
ids_n_max_rtw = list(map(lambda x: 1 if x == 0 else x ,ids_n_max_rtw))
ids_n_max_rtw

[1, 3, 1, 16, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1]

In [69]:
for ids_n,new_rtw in zip(ids_n_rtw,ids_n_max_rtw):
    index = df2[df2['id'] == ids_n].index[0]
    df2.set_value(index,'rtw',new_rtw)

In [70]:
df2[df2['id'].isin(ids_n_rtw)]

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet,rt_nik,all_nik_len
72368,410483415818199041,1386701847,imerkouri,"Ахахахахаха RT @nzlobin Действительно, несколь...",1,0,1,0,70202,65663,174,608,2013-12-10 22:57:27,1,nzlobin,1
78214,410718165023600640,1386757815,assimetrik2012,RT @riarip Абамой??? Абамой блядь! http://t.co...,1,0,1,0,7449,361,389,11,2013-12-11 14:30:15,1,riarip,1
96140,411022505189462016,1386830376,dmshardakov,RT @dmshardakov\n#ДеньКонституции продолжается...,1,0,1,0,586,185,182,1,2013-12-12 10:39:36,1,dmshardakov,1
101245,411090393829486592,1386846562,mucobyqejeg,"RT @kidukomypolu: хм, включила проигрыватель, ...",1,0,1,0,903,252,256,0,2013-12-12 15:09:22,1,kidukomypolu,1
28889,412635825609146368,1387215021,wirogeboses,"RT @hofequtamit: — Давай поспорим на $100, кто...",-1,0,3,0,1365,257,273,0,2013-12-16 21:30:21,1,hofequtamit,1
34482,413322070617911297,1387378635,reamagcaro,#RT @scufasaxim : Самый тяжёлый день- это #чет...,-1,0,1,0,7,8,10,0,2013-12-18 18:57:15,1,scufasaxim,1
34621,413330125283807232,1387380555,grovimebad,#RT @reamagcaro: #RT @maitiawreathter : Самый ...,-1,0,1,0,23,42,37,0,2013-12-18 19:29:15,1,reamagcaro,2
34622,413330169206550528,1387380566,carpgifttencoa,#RT @maitiawreathter : Самый тяжёлый день- это...,-1,0,1,0,50,100,77,0,2013-12-18 19:29:26,1,maitiawreathter,1
34703,413334183377793024,1387381523,kopfsuryfac,#RT @dubbcesssecli: #RT @maitiawreathter : Сам...,-1,0,2,0,7,40,48,0,2013-12-18 19:45:23,1,dubbcesssecli,2
34706,413334187073359872,1387381524,stowveruptcom,RT @dubbcesssecli: #RT @worldDG : Самый тяжёлы...,-1,0,1,0,7,39,78,0,2013-12-18 19:45:24,1,dubbcesssecli,2


In [71]:
for x, y in zip(ids_n_rtw,ids_n_max_rtw):
    print(x,y)

410483415818199041 1
412635825609146368 3
411090393829486592 1
417252268543864832 16
413322070617911297 1
413334187073359872 1
413330125283807232 1
411022505189462016 1
413334183377793024 2
413722067603701760 1
413723357188608000 2
413347322089586688 1
422611188350861312 1
410718165023600640 1
413330169206550528 1


Всё верно

In [72]:
df2[df2['id'].isin(ids_n_rtw)]['rtw'].value_counts()

1     11
2      2
3      1
16     1
Name: rtw, dtype: int64

данное правило даёт малое пополнение нашей выборки

###### финальная таблица

In [73]:
df1.rtw.value_counts()

0     189325
1        637
6          2
2          2
14         1
4          1
Name: rtw, dtype: int64

In [74]:
df1_new = df1.copy()

In [75]:
df1_new['rt_nik'] = df1_new.retweet.apply(lambda x: 0)
df1_new['all_nik_len'] = df1_new.retweet.apply(lambda x: 0)

In [76]:
df1_new = df1_new.append(df2[df2['id'].isin(ids_n_rtw)])

In [77]:
df1_new.rtw.value_counts()

0     189325
1        648
2          4
6          2
16         1
14         1
4          1
3          1
Name: rtw, dtype: int64

In [78]:
df1_new

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet,rt_nik,all_nik_len
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0,2013-12-06 14:32:07,0,0,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,2013-12-06 14:32:07,0,0,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1,2013-12-06 14:32:23,0,0,0
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0,2013-12-06 14:32:23,0,0,0
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0,0,0,5872,1387,1431,12,2013-12-06 14:32:24,0,0,0
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,0,0,0,263,5,36,0,2013-12-06 14:32:24,0,0,0
10,408906765896785920,1386325944,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,0,0,0,20098,619,38,1,2013-12-06 14:32:24,0,0,0
12,408906818287828993,1386325957,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,0,0,0,49346,481,123,21,2013-12-06 14:32:37,0,0,0
13,408906819084754944,1386325957,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,0,0,0,63,11,30,0,2013-12-06 14:32:37,0,0,0
14,408906820019712002,1386325957,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,0,0,0,14666,258,352,7,2013-12-06 14:32:37,0,0,0


In [79]:
df1_new.to_csv('df1_new.csv')

In [80]:
df1_new.columns

Index(['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'faw', 'stcount',
       'foll', 'frien', 'listcount', 'date_comf', 'retweet', 'rt_nik',
       'all_nik_len'],
      dtype='object')

In [81]:
y1_new = df1_new['rtw']
X1_new = df1_new[['id', 'date', 'name', 'text', 'type',  'stcount',
       'foll', 'frien', 'listcount', 'date_comf']]

In [82]:
X1_new

,id,date,name,text,type,stcount,foll,frien,listcount,date_comf
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,7569,62,61,0,2013-12-06 14:32:07
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,11825,59,31,2,2013-12-06 14:32:07
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,597,16,23,1,2013-12-06 14:32:23
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,40,6,16,0,2013-12-06 14:32:23
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,5872,1387,1431,12,2013-12-06 14:32:24
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,263,5,36,0,2013-12-06 14:32:24
10,408906765896785920,1386325944,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,20098,619,38,1,2013-12-06 14:32:24
12,408906818287828993,1386325957,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,49346,481,123,21,2013-12-06 14:32:37
13,408906819084754944,1386325957,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,63,11,30,0,2013-12-06 14:32:37
14,408906820019712002,1386325957,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,14666,258,352,7,2013-12-06 14:32:37


##### 2 вариант - исходного твита нет в нашей выборке (расширяем засчет ретвитов)

Будем брать ретвиты, в которых только 1-2 никнейма ( логика объеснена в (#) ), и преобразовывать их в первоначальные твиты  от автора (которого ретвитнули). 

Также назначаем твиту последнее значение количества ретвитов из максимума по последней дате (из всех твитов), либо по максимум из всего кол-ва ретвитов (потому что у ретвитов, количество ретвитов - количество ретвитов исходного твита)

In [83]:
df2.shape,df2[df2.all_nik_len<=2].shape

((36866, 16), (34555, 16))

In [84]:
cnt_dup_txt = df2[df2.all_nik_len<=2].text.value_counts()

In [85]:
cnt_dup_txt

RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT                                                                                           105
RT @Creative_Link: Дождались! Официальный трейлер "Иван Царевич и Серый Волк - 2". Смотреть всем: http://t.co/gmelSGMIuW Толи еще будет)           74
RT @Podslyshano: люблю зиму за то, что люди носом разблокируют телефоны, и такие оглядываются типа без палева)) а я то все вижу. да да.            53
RT @Creative_Link: Иван Царевич и Серый Волк - 2. Идем в кино? премьера 26 декабря))) http://t.co/gmelSGMIuW                                       47
RT @gigliliput: Все делишки сделаны :) http://t.co/2HeTC5wfba                                                                                      45
RT @ktvsktvs: Влюблен в слово "бывает", ибо оно везде уместно.\n— ты ебанат!\n— бывает!\n— У меня кот умер :(\n— Бывает!                           42
RT @ITetereva2: Прикольные котята)) http://t.co/0bkpaHM6ML                                          

In [86]:
name_orig = []
fin_rtw = []
text_orig = []
sentiment = []
for dup_text in  tqdm_notebook(cnt_dup_txt.index.values):
    
    name_orig.append(df2[df2.text == dup_text]['rt_nik'].values[0])
    # сравниваю максиммум ещё с 1, тк дб ретвит
    fin_rtw.append(max(df2[df2.text == dup_text].iloc[-1].rtw, max(df2[df2.text == dup_text].rtw),1))
    
    sentiment.append(df2[df2.text == dup_text]['type'].values[0])
    # убираем лишнее в твите, тк исходное сообщение было без идентификатора ретвита
    text_orig.append(re.sub('RT @[^\s]+ ', '', dup_text))
    

In [103]:
add_df = {'name':name_orig,'text':text_orig,'type':sentiment, 'rtw':fin_rtw}
len(name_orig),len(fin_rtw),len(cnt_dup_txt.index.values)

(29966, 29966, 29966)

In [104]:
add_df = pd.DataFrame(add_df)
add_df

,name,text,type,rtw
0,ARTEMKLYUSHIN,Кто ненавидит пробки ретвит((((( RT,-1,1381
1,CreativeLink,"Дождались! Официальный трейлер ""Иван Царевич и...",1,995
2,Podslyshano,"люблю зиму за то, что люди носом разблокируют ...",1,1068
3,CreativeLink,Иван Царевич и Серый Волк - 2. Идем в кино? пр...,1,976
4,gigliliput,Все делишки сделаны :) http://t.co/2HeTC5wfba,1,466
5,ktvsktvs,"Влюблен в слово ""бывает"", ибо оно везде уместн...",-1,1044
6,ITetereva2,Прикольные котята)) http://t.co/0bkpaHM6ML,1,250
7,supremefm,Хочу тату :( http://t.co/2LuWQRS5H3,-1,420
8,katunder1984,"Завтра День Конституции! ""Единая Россия"" прове...",1,423
9,craazyyymofo,"бабушка Луи успела снятся в фильме, но до прем...",-1,210


при проверке вручную (лучше просмотреть верхние профили)

видно, что кол-во ретвитов в сделанной таблице и кол-во ретвитов в твиттере отличается (в сделанной таблице больше) это происходит, тк часть людей удалили свои ретвиты + большое количество аккаунтов, которые есть в датасете удалены и ,как я понимаю, ретвиты таких аккаунтов убираются из кол-ва ретвитов твита

###### если использовать данную таблицу, для предсказания, то можно пользоваться только текстовыми признаками и сентиментом.


Если хочется работать с признаками пользователя, то можно по имени автора добавить последние по времени данные по друзьям, подписчикам и группам (но опять же, даты могут быть слишком разные поэтому я решил не исследовать с данным датасетом)!!!

In [105]:
len(set(add_df.name).intersection(set(df.name)))

8651

In [106]:
df[df.name.isin(set(add_df.name))].drop_duplicates('name',keep='last')

,id,date,name,text,type,rep,rtw,faw,stcount,foll,frien,listcount,date_comf,retweet
6,408906762436481024,1386325943,twinkleAYO,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",1,0,1,0,5169,58,43,2,2013-12-06 14:32:23,1
123,408908138381713408,1386326271,DrNickTo,"@upsguest Да да, так всё и будет. Кто где веч...",1,0,0,0,16983,116,58,6,2013-12-06 14:37:51,0
139,408908226097213440,1386326292,dudecusahywo,"Проходят мимо две девушки, а я как крикну как ...",1,0,0,0,399,177,186,0,2013-12-06 14:38:12,0
212,408909102094766080,1386326501,fywaweqeo,После введения инвайтов nnm-club похож на прир...,1,0,0,0,133,80,83,0,2013-12-06 14:41:41,0
322,408910343146987520,1386326797,Itisbelieved,"@Bumali2 Ну вот Мария, сам Премьер занялся про...",1,0,0,0,9171,490,399,7,2013-12-06 14:46:37,0
324,408910382473183232,1386326807,ElisabettStyles,@Crazy_weirdo_ собираюсь пойти на фильм только...,1,0,0,0,3636,115,200,0,2013-12-06 14:46:47,0
341,408910559380504576,1386326849,bucejimejak,Чтобы нормально поработать - нажмите сейчас кр...,1,0,0,0,15,4,4,0,2013-12-06 14:47:29,0
387,408911023697956864,1386326960,digger2912,"""Кто то в углу сидит и погибает от голода, а м...",1,0,0,0,193,17,9,0,2013-12-06 14:49:20,0
417,408911424816025601,1386327055,Steve119341,Чуть не кончил пока держал руки под горячей во...,1,0,0,0,3709,52,40,0,2013-12-06 14:50:55,0
469,408911862877532160,1386327159,AlinaSozonik,мы с Юлей нормально не можем вместе свалить))Н...,1,0,0,0,539,24,34,0,2013-12-06 14:52:39,0


###### финальная таблица

In [107]:
add_df.columns

Index(['name', 'text', 'type', 'rtw'], dtype='object')

In [108]:
df2_new = df1_new[add_df.columns]

In [109]:
df2_new = df2_new.append(add_df)

In [110]:
df2_new.reset_index(drop=True, inplace=True)
df2_new

,name,text,type,rtw
0,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0
1,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0
2,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0
3,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0
4,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0
5,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,0
6,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,0
7,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,0
8,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,0
9,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,0


In [111]:
df2_new.to_csv('df2_new.csv')

In [88]:
cnt_rtw = df2_new.rtw.value_counts()

In [89]:
sum(cnt_rtw[cnt_rtw.index>=1]),cnt_rtw[cnt_rtw.index==0].values[0]

(30624, 189325)

In [90]:
y2_new = df2_new['rtw']
X2_new = df2_new.drop('rtw',axis=1)

In [91]:
X2_new.columns

Index(['name', 'text', 'type'], dtype='object')

In [92]:
y2_new

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
          ..
219919     1
219920    11
219921     1
219922     2
219923     2
219924     2
219925     1
219926     1
219927     1
219928     1
219929     1
219930     1
219931     1
219932     1
219933     1
219934     1
219935     1
219936     4
219937     7
219938     1
219939     1
219940     2
219941     1
219942     1
219943     1
219944     1
219945     1
219946     1
219947     1
219948     1
Name: rtw, Length: 219949, dtype: int64

## Создание моделей

#### Сделаем дополнительные признаки

Что можно использовать в качестве входной информации? 
 - Среднее количество ретвитов на предыдущие твиты; (мало твитов по пользователю поэтому не делаю)
 - можно использовать содержание, какие-то темы более популярны, какие-то менее популярны;
 - можно использовать хештеги; (LabelEncoding)
 - количество подписчиков, информацию о них,
 - еще можно использовать время, когда написан твит. 

In [119]:
y1_new.head(10)

0     0
1     0
4     0
5     0
8     0
9     0
10    0
12    0
13    0
14    0
Name: rtw, dtype: int64

In [120]:
X1_new.head(10)

,id,date,name,text,type,stcount,foll,frien,listcount,date_comf
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,7569,62,61,0,2013-12-06 14:32:07
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,11825,59,31,2,2013-12-06 14:32:07
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,597,16,23,1,2013-12-06 14:32:23
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,40,6,16,0,2013-12-06 14:32:23
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,5872,1387,1431,12,2013-12-06 14:32:24
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,263,5,36,0,2013-12-06 14:32:24
10,408906765896785920,1386325944,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,20098,619,38,1,2013-12-06 14:32:24
12,408906818287828993,1386325957,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,49346,481,123,21,2013-12-06 14:32:37
13,408906819084754944,1386325957,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,63,11,30,0,2013-12-06 14:32:37
14,408906820019712002,1386325957,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,14666,258,352,7,2013-12-06 14:32:37


Дополнительные признаки выборка 1:
 - выделим час, день и месяц в котором был сделан твит
 - количество хэштегов
 - количество униграм в твите 
 - длина твита 

In [121]:
tknzr = TweetTokenizer()
def tokenize_ru(text):
    # firstly let's apply nltk tokenization
    
    text = text.replace("«", "").replace("»", "").replace(',','')
    tokens = tknzr.tokenize(text)
    return tokens

In [122]:
X1_new['hour'] = X1_new.date_comf.apply(lambda x: pd.to_datetime(x).hour)
X1_new['day'] = X1_new.date_comf.apply(lambda x: pd.to_datetime(x).day)
X1_new['month'] = X1_new.date_comf.apply(lambda x: pd.to_datetime(x).month)
X1_new['cnt_htg'] = X1_new.text.apply(lambda x: len(re.findall('#[^\s]+', x)))
X1_new['cnt_uni'] = X1_new.text.apply(lambda x: len(tokenize_ru(x)))
X1_new['cnt_chr'] = X1_new.text.apply(lambda x: len(x))

In [123]:
X1_new.head(10)

,id,date,name,text,type,stcount,foll,frien,listcount,date_comf,hour,day,month,cnt_htg,cnt_uni,cnt_chr
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,7569,62,61,0,2013-12-06 14:32:07,14,6,12,0,18,99
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,11825,59,31,2,2013-12-06 14:32:07,14,6,12,0,15,71
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,597,16,23,1,2013-12-06 14:32:23,14,6,12,0,17,125
5,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,40,6,16,0,2013-12-06 14:32:23,14,6,12,0,14,66
8,408906764608749568,1386325944,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,5872,1387,1431,12,2013-12-06 14:32:24,14,6,12,0,6,51
9,408906765841870848,1386325944,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,263,5,36,0,2013-12-06 14:32:24,14,6,12,2,10,86
10,408906765896785920,1386325944,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,20098,619,38,1,2013-12-06 14:32:24,14,6,12,0,19,71
12,408906818287828993,1386325957,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,49346,481,123,21,2013-12-06 14:32:37,14,6,12,0,8,62
13,408906819084754944,1386325957,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,63,11,30,0,2013-12-06 14:32:37,14,6,12,0,15,58
14,408906820019712002,1386325957,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,14666,258,352,7,2013-12-06 14:32:37,14,6,12,0,25,133


In [125]:
np.max(X1_new)

id                                          425138595251625984
date                                                1390195914
name                                                  zzzzz245
text         ������������ - А помните как раньше с фруктов ...
type                                                         1
stcount                                                1138639
foll                                                   1582807
frien                                                   388311
listcount                                                16915
date_comf                                  2014-01-20 09:31:54
hour                                                        23
day                                                         31
month                                                       12
cnt_htg                                                     12
cnt_uni                                                     60
cnt_chr                                                

In [126]:
X1_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189983 entries, 0 to 95161
Data columns (total 16 columns):
id           189983 non-null int64
date         189983 non-null int64
name         189983 non-null object
text         189983 non-null object
type         189983 non-null int64
stcount      189983 non-null int64
foll         189983 non-null int64
frien        189983 non-null int64
listcount    189983 non-null int64
date_comf    189983 non-null datetime64[ns]
hour         189983 non-null int64
day          189983 non-null int64
month        189983 non-null int64
cnt_htg      189983 non-null int64
cnt_uni      189983 non-null int64
cnt_chr      189983 non-null int64
dtypes: datetime64[ns](1), int64(13), object(2)
memory usage: 24.6+ MB


In [259]:
X1 = X1_new[['name', 'text', 'type', 'stcount', 'foll', 'frien',
       'listcount','hour', 'day', 'month', 'cnt_htg', 'cnt_uni',
       'cnt_chr']].copy()

проведем нормализацию данных (практически без разницы каким методом)

In [93]:
def Normalization(series):
    return (series - series.mean()) / series.std()

In [94]:
def MinMax(series):
    return (series - series.min()) / (series.max() - series.min()) 

In [262]:
le = LabelEncoder()

In [263]:
X1['stcount'] = MinMax(X1.stcount)
X1['foll'] = MinMax(X1.foll)
X1['frien'] = MinMax(X1.frien)
X1['listcount'] = MinMax(X1.listcount)
X1['hour'] = MinMax(X1.hour)
X1['day'] = MinMax(X1.day)
X1['month'] = MinMax(X1.month)

X1['cnt_htg'] = X1.cnt_htg/max(X1.cnt_htg)
X1['cnt_uni'] = MinMax(X1.cnt_uni)
X1['cnt_chr'] = MinMax(X1.cnt_chr)

X1['name'] = le.fit_transform(X1['name'])
X1['name'] = X1['name']/max(X1.name)

Сделаем препроцессинг текстовых признаков 

In [265]:

prep_tf_idf = ColumnTransformer(
                    transformers=[('ntext', TfidfVectorizer(), 'text')])

X1_chng_tf_idf = prep_tf_idf.fit_transform(X1)


Сожмем пространство TF-IDF/BoW признаков потому что если будем работать со всеми, то не хватит памяти

In [266]:
svd = TruncatedSVD(n_components=50, n_iter=5, random_state=1712)


X1_chng_tf_idf_svd = svd.fit_transform(X1_chng_tf_idf)


На TF-IDF работает лучше, чем на BoW

### Сделаем сбанасированную выборку

In [267]:
smote = SMOTE(random_state=1712)
adasyn = ADASYN(random_state=1712)

In [268]:
y1_new.value_counts()

0     189325
1        648
2          4
6          2
16         1
14         1
4          1
3          1
Name: rtw, dtype: int64

Объединим значения ретвитов большие 0, иначе не удается сгенерировать сбалансированные данные

In [269]:
y1_unit = y1_new.copy()
y1_unit[y1_unit>0] = 1

In [270]:
y1_unit.value_counts()

0    189325
1       658
Name: rtw, dtype: int64

In [271]:
X1_smote, y1_smote = smote.fit_resample(X1_chng_tf_idf_svd, y1_unit)
print(sorted(Counter(y_smote).items()))

[(0, 189325), (1, 189325)]


In [272]:
X1_adasyn, y1_adasyn = adasyn.fit_resample(X1_chng_tf_idf_svd, y1_unit)
print(sorted(Counter(y_adasyn).items()))

[(0, 189325), (1, 189327)]


In [273]:
x1_sm_train, x1_sm_test, y1_sm_train, y1_sm_test = train_test_split(X1_smote, y1_smote, test_size=0.2, random_state=1712)

In [274]:
x1_ada_train, x1_ada_test, y1_ada_train, y1_ada_test = train_test_split(X1_adasyn, y1_adasyn, test_size=0.2, random_state=1712)

### Обучение 

##### SMOTE

In [275]:
pipeline = Pipeline([
    ('clf', LogisticRegression()),
])
parameters = [
    {
        'clf': [Ridge(),Lasso()],
        'clf__alpha': np.logspace(-4, 1, 6),
    },
    {
        'clf': [SGDRegressor(random_state=1712)],
#         'clf__alpha': np.logspace(-4, 0, 5),
#         'clf__penalty': ['l2', 'l1'],
        'clf__eta0':np.logspace(-4, -2, 2),
    },
    
   
#      {  'clf': [RandomForestRegressor(criterion='mae',random_state=1712)],
#         'clf__max_depth': [5,10,20],
# #         'clf__min_samples_split':  [2, 5, 10],
#         'clf__n_estimators': [10,50]#[10,50, 100, 500],
#         }
]

grid_logit = GridSearchCV(pipeline, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1,verbose=1)
grid_logit.fit(x1_sm_train, y1_sm_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    7.5s finished


GridSearchCV(cv=5, estimator=Pipeline(steps=[('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'clf': [Ridge(alpha=0.0001), Lasso()],
                          'clf__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])},
                         {'clf': [SGDRegressor(random_state=1712)],
                          'clf__eta0': array([0.0001, 0.01  ])}],
             scoring='neg_mean_absolute_error', verbose=1)

In [276]:
print("Best parameters set:")
best_parameters = grid_logit.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	clf: Ridge(alpha=0.0001)
	clf__alpha: 0.0001
	clf__copy_X: True
	clf__fit_intercept: True
	clf__max_iter: None
	clf__normalize: False
	clf__random_state: None
	clf__solver: 'auto'
	clf__tol: 0.001
	memory: None
	steps: [('clf', Ridge(alpha=0.0001))]
	verbose: False


In [277]:
grid_logit.best_score_

-0.4491154938372327

In [278]:
print("Для SMOTE")
print("Train:",mean_absolute_error(y1_sm_train, grid_logit.predict(x1_sm_train)),'|',"Test:",mean_absolute_error(y1_sm_test, grid_logit.predict(x1_sm_test)))

Для SMOTE
Train: 0.44905313384164713 | Test: 0.450849592866623


##### ADASYN

In [279]:
pipeline = Pipeline([
    ('clf', LogisticRegression()),
])
parameters = [
    {
        'clf': [Ridge(),Lasso()],
        'clf__alpha': np.logspace(-4, 1, 6),
    },
    {
        'clf': [SGDRegressor(random_state=1712)],
# К сожалению, не хватило времени попробовать
#         'clf__alpha': np.logspace(-4, 0, 5),
#         'clf__penalty': ['l2', 'l1'],
        'clf__eta0':np.logspace(-4, -2, 2),
    },
    
# К сожалению, не хватило времени попробовать   
#      {  'clf': [RandomForestRegressor(criterion='mae',random_state=1712)],
#         'clf__max_depth': [5,10,20],
# #         'clf__min_samples_split':  [2, 5, 10],
#         'clf__n_estimators': [10,50]#[10,50, 100, 500],
#         }
]

grid_logit = GridSearchCV(pipeline, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1,verbose=1)
grid_logit.fit(x2_sm_train, y2_sm_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    7.6s finished


GridSearchCV(cv=5, estimator=Pipeline(steps=[('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'clf': [Ridge(alpha=0.0001), Lasso()],
                          'clf__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])},
                         {'clf': [SGDRegressor(random_state=1712)],
                          'clf__eta0': array([0.0001, 0.01  ])}],
             scoring='neg_mean_absolute_error', verbose=1)

In [280]:
print("Best parameters set:")
best_parameters = grid_logit.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	clf: Ridge(alpha=0.0001)
	clf__alpha: 0.0001
	clf__copy_X: True
	clf__fit_intercept: True
	clf__max_iter: None
	clf__normalize: False
	clf__random_state: None
	clf__solver: 'auto'
	clf__tol: 0.001
	memory: None
	steps: [('clf', Ridge(alpha=0.0001))]
	verbose: False


In [281]:
grid_logit.best_score_

-0.4500506452782954

In [282]:
print("Для Adasyn")
print("Train:",mean_absolute_error(y1_sm_train, grid_logit.predict(x1_sm_train)),'|',"Test:",mean_absolute_error(y1_sm_test, grid_logit.predict(x1_sm_test)))

Для Adasyn
Train: 0.44958429515110093 | Test: 0.4513214683424036


Как видим различные способы балансировки данных, дают примерно одинаковое итоговое значение значение меры качества на train & test.

Лучший результат **0.45** (на train & test) по метрике качества MAE был достигнут с помощью модели Ridge на сбалансированной выборке с помощью **SMOTE**.

Далее в связи с недостатком времени и результатами выше воспользуюсь только балансировкой с помощью SMOTE.

## Выборка 2

In [95]:
y2_new.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: rtw, dtype: int64

In [96]:
X2_new.head(10)

,name,text,type
0,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1
1,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1
2,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1
3,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1
4,grishintv,Поприветствуем моего нового читателя @Alexey17...,1
5,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1
6,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1
7,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1
8,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1
9,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1


In [97]:
tknzr = TweetTokenizer()
def tokenize_ru(text):
    # firstly let's apply nltk tokenization
    
    text = text.replace("«", "").replace("»", "").replace(',','')
    tokens = tknzr.tokenize(text)
    return tokens

In [98]:
X2_new['cnt_htg'] = X2_new.text.apply(lambda x: len(re.findall('#[^\s]+', x)))
X2_new['cnt_uni'] = X2_new.text.apply(lambda x: len(tokenize_ru(x)))
X2_new['cnt_chr'] = X2_new.text.apply(lambda x: len(x))

In [99]:
X2_new.head(10)

,name,text,type,cnt_htg,cnt_uni,cnt_chr
0,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,18,99
1,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,15,71
2,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,17,125
3,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,14,66
4,grishintv,Поприветствуем моего нового читателя @Alexey17...,1,0,6,51
5,alina_612,Теперь у меня есть частичка Сиднея :) #Sydney ...,1,2,10,86
6,anvaharpew,Люблю маму и папу!!!!а в остальное я так...-вл...,1,0,19,71
7,Dane_Rider,@MrsRourke_s_tit @_vivante @dyu_bryun так было...,1,0,8,62
8,Eugenia_lev,@Kruglova_Julia_ дааааа))\nТы... Ты... Ты и т...,1,0,15,58
9,Vitaliy_Ivanov,"@Alex_Shvarz :)) О, нет. Вы ведь всё равно обз...",1,0,25,133


In [100]:
X2_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219949 entries, 0 to 219948
Data columns (total 6 columns):
name       219949 non-null object
text       219949 non-null object
type       219949 non-null int64
cnt_htg    219949 non-null int64
cnt_uni    219949 non-null int64
cnt_chr    219949 non-null int64
dtypes: int64(4), object(2)
memory usage: 11.7+ MB


In [101]:
X2 = X2_new.copy()
le = LabelEncoder()

In [102]:
X2['cnt_htg'] = X2.cnt_htg/max(X2.cnt_htg)

X2['cnt_uni'] = MinMax(X2.cnt_uni)
X2['cnt_chr'] = MinMax(X2.cnt_chr)

X2['name'] = le.fit_transform(X2['name'])
X2['name'] = X2['name']/max(X2.name)

Сделаем препроцессинг текстовых признаков 

In [103]:

prep_tf_idf = ColumnTransformer(
                    transformers=[('ntext', TfidfVectorizer(), 'text')])

X2_chng_tf_idf = prep_tf_idf.fit_transform(X2)

Сожмем пространство TF-IDF/BoW признаков потому что если будем работать со всеми, то не хватит памяти

In [104]:
svd = TruncatedSVD(n_components=50, n_iter=5, random_state=1712)


X2_chng_tf_idf_svd = svd.fit_transform(X2_chng_tf_idf)


### Сделаем сбанасированную выборку

In [105]:
smote = SMOTE(random_state=1712)
adasyn = ADASYN(random_state=1712)

Объединим значения ретвитов большие 0, иначе не удается сгенерировать сбалансированные данные

In [106]:
y2_new.value_counts()

0        189325
1         20908
2          3599
3          1707
4           956
5           602
6           433
7           301
8           227
9           208
10          166
12          136
11          113
13           83
14           74
16           67
15           64
17           60
18           54
19           36
22           33
24           31
26           29
23           26
25           25
20           24
21           23
28           22
32           21
27           19
          ...  
472           1
89            1
354           1
1381          1
160           1
288           1
416           1
357           1
229           1
162           1
101           1
163           1
995           1
292           1
420           1
98            1
305           1
423           1
607           1
169           1
223           1
170           1
478           1
1068          1
222           1
173           1
174           1
221           1
474           1
13817         1
Name: rtw, Length: 212, 

Объединим значения ретвитов большие 0 по разным группам, иначе не удается сгенерировать сбалансированные данные 

In [118]:
y2_unit = y2_new.copy()

In [119]:
y2_new.value_counts()

0        189325
1         20908
2          3599
3          1707
4           956
5           602
6           433
7           301
8           227
9           208
10          166
12          136
11          113
13           83
14           74
16           67
15           64
17           60
18           54
19           36
22           33
24           31
26           29
23           26
25           25
20           24
21           23
28           22
32           21
27           19
          ...  
472           1
89            1
354           1
1381          1
160           1
288           1
416           1
357           1
229           1
162           1
101           1
163           1
995           1
292           1
420           1
98            1
305           1
423           1
607           1
169           1
223           1
170           1
478           1
1068          1
222           1
173           1
174           1
221           1
474           1
13817         1
Name: rtw, Length: 212, 

In [120]:
y2_unit[(y2_unit>0)&(y2_unit<4)] = 1
y2_unit[(y2_unit>=4)&(y2_unit<10)] = 2
y2_unit[y2_unit>=10] = 3

In [121]:
y2_unit.value_counts()

0    189325
1     26214
2      2727
3      1683
Name: rtw, dtype: int64

In [122]:
X2_smote, y2_smote = smote.fit_resample(X2_chng_tf_idf_svd, y2_unit)

In [123]:
print(sorted(Counter(y2_smote).items()))

[(0, 189325), (1, 189325), (2, 189325), (3, 189325)]


In [126]:
x2_sm_train, x2_sm_test, y2_sm_train, y2_sm_test = train_test_split(X2_smote, y2_smote, test_size=0.2, random_state=1712)

### Обучение 

In [127]:
pipeline = Pipeline([
    ('clf', LogisticRegression()),
])
parameters = [
    {
        'clf': [Ridge(),Lasso()],
        'clf__alpha': np.logspace(-4, 1, 6),
    },
    {
        'clf': [SGDRegressor(random_state=1712)],
# К сожалению, не хватило времени попробовать
#         'clf__alpha': np.logspace(-4, 0, 5),
#         'clf__penalty': ['l2', 'l1'],
        'clf__eta0':np.logspace(-4, -2, 2),
    },
    
# К сожалению, не хватило времени попробовать   
#      {  'clf': [RandomForestRegressor(criterion='mae',random_state=1712)],
#         'clf__max_depth': [5,10,20],
# #         'clf__min_samples_split':  [2, 5, 10],
#         'clf__n_estimators': [10,50]#[10,50, 100, 500],
#         }
]

grid_logit = GridSearchCV(pipeline, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1,verbose=1)
grid_logit.fit(x2_sm_train, y2_sm_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   17.9s finished


GridSearchCV(cv=5, estimator=Pipeline(steps=[('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'clf': [Ridge(alpha=0.0001), Lasso()],
                          'clf__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])},
                         {'clf': [SGDRegressor(random_state=1712)],
                          'clf__eta0': array([0.0001, 0.01  ])}],
             scoring='neg_mean_absolute_error', verbose=1)

In [128]:
grid_logit.best_score_

-0.8809196824520755

In [129]:
print("Best parameters set:")
best_parameters = grid_logit.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	clf: Ridge(alpha=0.0001)
	clf__alpha: 0.0001
	clf__copy_X: True
	clf__fit_intercept: True
	clf__max_iter: None
	clf__normalize: False
	clf__random_state: None
	clf__solver: 'auto'
	clf__tol: 0.001
	memory: None
	steps: [('clf', Ridge(alpha=0.0001))]
	verbose: False


In [130]:
print("Для SMOTE")
print("Train:",mean_absolute_error(y2_sm_train, grid_logit.predict(x2_sm_train)),'|',"Test:",mean_absolute_error(y2_sm_test, grid_logit.predict(x2_sm_test)))

Для SMOTE
Train: 0.8808371957778002 | Test: 0.8828042121418228


###### Посмотрим случай когда только 2 класса, как в 1 выборке

In [131]:
y2_new.value_counts()

0        189325
1         20908
2          3599
3          1707
4           956
5           602
6           433
7           301
8           227
9           208
10          166
12          136
11          113
13           83
14           74
16           67
15           64
17           60
18           54
19           36
22           33
24           31
26           29
23           26
25           25
20           24
21           23
28           22
32           21
27           19
          ...  
472           1
89            1
354           1
1381          1
160           1
288           1
416           1
357           1
229           1
162           1
101           1
163           1
995           1
292           1
420           1
98            1
305           1
423           1
607           1
169           1
223           1
170           1
478           1
1068          1
222           1
173           1
174           1
221           1
474           1
13817         1
Name: rtw, Length: 212, 

Объединим значения ретвитов большие 0, иначе не удается сгенерировать сбалансированные данные 

In [133]:
y2_unit = y2_new.copy()

In [134]:
y2_new.value_counts()

0        189325
1         20908
2          3599
3          1707
4           956
5           602
6           433
7           301
8           227
9           208
10          166
12          136
11          113
13           83
14           74
16           67
15           64
17           60
18           54
19           36
22           33
24           31
26           29
23           26
25           25
20           24
21           23
28           22
32           21
27           19
          ...  
472           1
89            1
354           1
1381          1
160           1
288           1
416           1
357           1
229           1
162           1
101           1
163           1
995           1
292           1
420           1
98            1
305           1
423           1
607           1
169           1
223           1
170           1
478           1
1068          1
222           1
173           1
174           1
221           1
474           1
13817         1
Name: rtw, Length: 212, 

In [135]:
y2_unit[y2_unit>0] = 1

In [136]:
y2_unit.value_counts()

0    189325
1     30624
Name: rtw, dtype: int64

In [137]:
X2_smote, y2_smote = smote.fit_resample(X2_chng_tf_idf_svd, y2_unit)

In [138]:
print(sorted(Counter(y2_smote).items()))

[(0, 189325), (1, 189325)]


In [139]:
x2_sm_train, x2_sm_test, y2_sm_train, y2_sm_test = train_test_split(X2_smote, y2_smote, test_size=0.2, random_state=1712)

### Обучение 

In [140]:
pipeline = Pipeline([
    ('clf', LogisticRegression()),
])
parameters = [
    {
        'clf': [Ridge(),Lasso()],
        'clf__alpha': np.logspace(-4, 1, 6),
    },
    {
        'clf': [SGDRegressor(random_state=1712)],
# К сожалению, не хватило времени попробовать
#         'clf__alpha': np.logspace(-4, 0, 5),
#         'clf__penalty': ['l2', 'l1'],
        'clf__eta0':np.logspace(-4, -2, 2),
    },
    
# К сожалению, не хватило времени попробовать   
#      {  'clf': [RandomForestRegressor(criterion='mae',random_state=1712)],
#         'clf__max_depth': [5,10,20],
# #         'clf__min_samples_split':  [2, 5, 10],
#         'clf__n_estimators': [10,50]#[10,50, 100, 500],
#         }
]

grid_logit = GridSearchCV(pipeline, parameters, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1,verbose=1)
grid_logit.fit(x2_sm_train, y2_sm_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    8.8s finished


GridSearchCV(cv=5, estimator=Pipeline(steps=[('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'clf': [Ridge(alpha=0.0001), Lasso()],
                          'clf__alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])},
                         {'clf': [SGDRegressor(random_state=1712)],
                          'clf__eta0': array([0.0001, 0.01  ])}],
             scoring='neg_mean_absolute_error', verbose=1)

In [141]:
grid_logit.best_score_

-0.4640128586933804

In [142]:
print("Best parameters set:")
best_parameters = grid_logit.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	clf: Ridge(alpha=0.0001)
	clf__alpha: 0.0001
	clf__copy_X: True
	clf__fit_intercept: True
	clf__max_iter: None
	clf__normalize: False
	clf__random_state: None
	clf__solver: 'auto'
	clf__tol: 0.001
	memory: None
	steps: [('clf', Ridge(alpha=0.0001))]
	verbose: False


In [143]:
print("Для SMOTE")
print("Train:",mean_absolute_error(y2_sm_train, grid_logit.predict(x2_sm_train)),'|',"Test:",mean_absolute_error(y2_sm_test, grid_logit.predict(x2_sm_test)))

Для SMOTE
Train: 0.46393097886168727 | Test: 0.4639673789453345


**Итог:**
 - 1 выборка дает лучший результат, на выборке сбалансированной по 2 таргетам
 - 2 выборка может использоваться на сбалансированной выборке по большему количеству таргетов
 
Возможные улучшения:
 - проверить на исходных данных, как модель делает предсказание и померить качество
 - в 1 выборку можно добавить признаки связанные с пользователем:
  - количество ретвитов в прошлых сообщениях
  - частота твитов
 